# Project 3 for CarND: Use Deep Learning to Clone Driving Behavior

## Objectives

- Use the same architecture as in 2.3, 2.6 and 3.1 to drive a car in the simulator

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Lambda, Cropping2D
from keras.utils import np_utils

Using TensorFlow backend.


<img align="left" src="files/new.png"> **Load data** in the driving_log, delete the path to the file

In [2]:
df_data = pd.read_csv("./driving_log.csv", names = ["center","left","right","steering","throttle","break","speed"])

<img align="left" src="files/new.png"> **Load images** add the path in front of the file name. Load the image and the mirrored image, doubling the data

In [3]:
x = []
y = []
for i, file_name in enumerate(df_data["center"]):
    img = mpimg.imread("./IMG/" + file_name)
    x.append(img)
    y.append(df_data["steering"][i])
    img = np.fliplr(img)
    x.append(img)
    y.append(-df_data["steering"][i])   


x = np.array(x)
y = np.array(y)

<img align="left" src="files/modified.png"> **Model** bring the images to 32x32 by Cropping2D and MaxPooling2D. The output is one variable, without the final softmax since it is a regression

In [4]:
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((60,4), (0,0)), input_shape=(160,320,3)))

model.add(MaxPooling2D(pool_size=(3,10)))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 5, 5, activation='relu', border_mode='same'))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')

<img align="left" src="files/old.png"> **Train**

In [5]:
model.fit(x, y, batch_size=128, nb_epoch=50, verbose=0)

<img align="left" src="files/new.png"> **Save the Model**

In [6]:
model.save('model.h5')